In [136]:
import os
import sys

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import datetime
import numpy as np
from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import astropy
import data_loader
import matplotlib.pyplot as plt

import importlib

pdata_folder = os.path.abspath(r"./../processed_data/chorus_neural_network/")

importlib.reload(data_loader)

%matplotlib qt

In [137]:
year = 2019
sat = "b"

density_folder = os.path.join(pdata_folder, "STAGE_0", "ELECTRON_DENSITY_DATA_PREPROCESSED")

start = datetime.datetime(year=year, month=1, day=1, tzinfo=datetime.UTC)
end = datetime.datetime(year=year + 1, month=1, day=1, tzinfo=datetime.UTC)

density_data = data_loader.load_raw_data_from_config(
    id=["RBSP", "EMFISIS", "L4", "DENSITY"],
    start=start,
    end=end,
    satellite=sat,
    root_data_dir="/project/rbsp/data/",
    use_config_keys_in_subdir=False,
)

density = density_data["density"]
epoch = density_data["Epoch"]

valid_data = (0 < density) & np.isfinite(density) & (0 < epoch) & np.isfinite(epoch)

density = density[valid_data]
epoch = epoch[valid_data]

In [138]:
unix_time = cdfepoch.unixtime(epoch)

In [139]:
evenly_spaced_minutes = np.arange(start.timestamp(), end.timestamp() + 60, step=60)

smoothed_density = []

for chunk in evenly_spaced_minutes:

    TIME_RANGE = np.searchsorted(a=unix_time, v=[(chunk - 300), (chunk + 300)])

    smoothed_density.append(np.nanmedian(density[TIME_RANGE[0] : TIME_RANGE[-1]]))

/home/jddoucette/Research_Tools/ResearchPy/lib64/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [140]:
np.savez(
    file=os.path.join(density_folder, f"RBSP_{sat.upper()}_OBSERVED_DENSITY_{year}.npz"),
    UNIX_TIME=np.asarray(evenly_spaced_minutes),
    DENSITY=np.asarray(smoothed_density)
)

In [141]:
plt.semilogy(unix_time, density)
plt.semilogy(np.asarray(evenly_spaced_minutes), np.asarray(smoothed_density))